# Multimodal Patch Based networks

The following model expands on the a patch based architectures by combining feature maps of both MRI and PET modalities using attention based mechanisms. As a part of the ablation study, we also train a model without attention based mechanisms. There are three classifcation stages for the final multimodal attention-based model: 
- <b>Patch Feature Extraction:</b> 
    - 27 ResNet models are trained on patches from each patch location to extract local features.
- <b>Multimodal Attention:</b>
    - For each patch location, we train a model which combines the PET and MRI feature maps at that location using the corresponding patch models.  Multihead attention is used to capture the relationships between both modalities.
- <b>Patch fusion:</b>
    - Feature maps of the 27 trained attention-patch models are concatenated and used as inputs to a final model for global classification

## Importing Libraries

In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import os
import gc
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler

 
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import *
import nibabel as nb
import torchio as tio

from models import (cnn, C3DNet, resnet, ResNetV2, ResNeXt, ResNeXtV2, WideResNet, PreActResNet,
        EfficientNet, DenseNet, ShuffleNet, ShuffleNetV2, SqueezeNet, MobileNet, MobileNetV2)




## Read in images

In [2]:
# Subject IDs who are progressive normal cognition
PNC = pd.read_pickle('PNC.pkl')

# Subject IDs who are stable normal cognition
SNC = pd.read_pickle('SNC.pkl')

In [3]:
# Create datasets

def read_image_data(input_path):
    X_train = []
    y_train = []
    ids = []
    for filename in sorted(os.listdir(input_path)):
        file = os.path.join(input_path, filename)
        img_file = nb.load(file)
        img = img_file.get_fdata()
        X_train.append(img)
        ids.append(filename)
        # Progressive normal cognition target class 1
        if filename[0:8] in np.array(PNC):
            y_train.append(1)
        # Stable normal cognition class 0
        else:
            y_train.append(0)

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    # Reshape X_train to include channel dimension
    X_train = X_train.reshape(X_train.shape + (1,))
    return X_train, y_train, ids

X_MRI, y_MRI, ids_MRI = read_image_data("E:/Work/Processed_MRI/2.MNI_Registration")
X_PET, y_PET, ids_PET = read_image_data("E:/Work/Processed_PIB/4.MNI_Registered")


## Creating training and test sets for images

In [4]:
# 0.6 - 0.2 - 0.2 train-val-test split
X_train_MRI, X_test_MRI, X_train_PET, X_test_PET, ids_train, ids_test, y_train, y_test = train_test_split(X_MRI, 
                                                                                     X_PET, ids_MRI, 
                                                                                     y_MRI, test_size=0.2, 
                                                                                     random_state=101, stratify=y_MRI)

X_train_MRI, X_val_MRI, X_train_PET, X_val_PET, ids_train, ids_val, y_train, y_val = train_test_split(X_train_MRI, X_train_PET, 
                                                                                      ids_train, y_train, 
                                                                                      test_size=0.25, random_state=101,
                                                                                      stratify=y_train)

In [5]:
# Convert datasets to tensor format, with channel first
train_x_MRI = torch.from_numpy(X_train_MRI).float().permute(0,4,1,2,3)
train_x_PET = torch.from_numpy(X_train_PET).float().permute(0,4,1,2,3)
train_y = torch.from_numpy(y_train).float()

val_x_MRI = torch.from_numpy(X_val_MRI).float().permute(0,4,1,2,3)
val_x_PET = torch.from_numpy(X_val_PET).float().permute(0,4,1,2,3)
val_y = torch.from_numpy(y_val).float()

test_x_MRI = torch.from_numpy(X_test_MRI).float().permute(0,4,1,2,3)
test_x_PET = torch.from_numpy(X_test_PET).float().permute(0,4,1,2,3)
test_y = torch.from_numpy(y_test).float()

## Perform data augmentation to increase training set size

In [ ]:
def perform_augmentation(dataset, seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    # Define transformations
    training_transform = tio.Compose([
        tio.RandomAffine(),
        tio.RandomElasticDeformation(),
        tio.RandomFlip()
    ])
    augmented_dataset = torch.clone(dataset) 
    for i in range(len(augmented_dataset)):
        augmented_dataset[i] = training_transform(augmented_dataset[i])
    return augmented_dataset

orig_train_x_MRI = torch.clone(train_x_MRI)
orig_train_x_PET = torch.clone(train_x_PET)
orig_train_y = torch.clone(train_y)
for seed in [1,101,42]:
    # Apply transformations and create augmented training set
    augmented_train_MRI = perform_augmentation(orig_train_x_MRI, seed)
    augmented_train_PET = perform_augmentation(orig_train_x_PET, seed)
    
    
    # Concatenate training and augmented training datasets
    train_x_MRI = torch.cat((train_x_MRI, augmented_train_MRI), 0)
    train_x_PET = torch.cat((train_x_PET, augmented_train_PET), 0)
    train_y = torch.cat((train_y, orig_train_y), 0)

# Removing blank space around brain to make dimensions even so we can divide image into uniform patches
train_x_MRI = train_x_MRI[:, :, 0:88, 0:108, 0:88]
train_x_PET = train_x_PET[:, :, 0:88, 0:108, 0:88]   

In [6]:
# Load augmented dataset (for reruns)
train_x_MRI = torch.load("train_x_MRI.pkl")
train_x_PET = torch.load("train_x_PET.pkl")
train_y = torch.load("train_y.pkl")

## Divide images into patches

In [7]:
# For each sample, divide images into 27 uniform 3x3x3 patches of size 44x54x44 with 50% overlap
# Then create 27 training datasets for each patch location 
# e.g a data set for patches of each subject in the top left corner, ..., a dataset for patches of each subject in the middle

def create_MRI_PET_patches(images, dim, labels):
    # Create datasets for each patch location
    datasets_MRI = [[] for _ in range(27)]
    datasets_PET = [[] for _ in range(27)]
    datasets_MRI_PET = [[] for _ in range(27)]
    # Find the starting locations of each patch
    starting_points = []
    for height_stride in range(3):
        for width_stride in range(3):
            for depth_stride in range(3):
                    start = (0 + height_stride*dim[1]//4, 0 + width_stride*dim[2]//4, 0 + depth_stride*dim[3]//4)
                    starting_points.append(start)

    # For each image
    for i in range(len(images[0])):
        # Create patch from every starting point
        for j in range(len(starting_points)):
            start_pt = starting_points[j]
            patch_MRI = images[0][i][:, start_pt[0]:start_pt[0] + dim[1]//2, 
                                      start_pt[1]:start_pt[1] + dim[2]//2, 
                                      start_pt[2]:start_pt[2] + dim[3]//2]
            
            patch_PET = images[1][i][:, start_pt[0]:start_pt[0] + dim[1]//2, 
                                      start_pt[1]:start_pt[1] + dim[2]//2, 
                                      start_pt[2]:start_pt[2] + dim[3]//2]
            datasets_MRI[j].append(patch_MRI)
            datasets_PET[j].append(patch_PET)
           
            
    # For each patch location, stack patches from every sample samples into a tensor as a Tensor Dataset
    for i in range(27):
        datasets_MRI[i] = torch.stack(datasets_MRI[i])
        datasets_PET[i] = torch.stack(datasets_PET[i]) 
        datasets_MRI_PET[i] = torch.utils.data.TensorDataset(datasets_MRI[i], datasets_PET[i], labels)
    return datasets_MRI_PET


In [8]:
# Create datasets for each patch location

dim = train_x_MRI[0].shape
patch_train_datasets_MRI_PET = create_MRI_PET_patches((train_x_MRI, train_x_PET), dim, train_y)
patch_val_datasets_MRI_PET = create_MRI_PET_patches((val_x_MRI, val_x_PET), dim, val_y)
patch_test_datasets_MRI_PET = create_MRI_PET_patches((test_x_MRI, test_x_PET), dim, test_y)


# Create data loaders for each patch location dataset
patch_train_loaders_MRI_PET = []
patch_val_loaders_MRI_PET = []
patch_test_loaders_MRI_PET = []

In [9]:
batch_size = 16
for i in range(27):
    patch_train_loaders_MRI_PET.append(torch.utils.data.DataLoader(patch_train_datasets_MRI_PET[i], 
                                                                  batch_size = batch_size, shuffle = True))
    patch_val_loaders_MRI_PET.append(torch.utils.data.DataLoader(patch_val_datasets_MRI_PET[i], 
                                                                  batch_size = batch_size, shuffle = False))
    patch_test_loaders_MRI_PET.append(torch.utils.data.DataLoader(patch_test_datasets_MRI_PET[i], 
                                                                  batch_size = batch_size, shuffle = False))

In [10]:
# Define device being used for model training
if torch.cuda.is_available(): 
    dev = "cuda:0" 
else: 
    dev = "cpu" 
device = torch.device(dev) 

## Model training

In [11]:
def train_model(model, train_loader, optimiser, error, scheduler, feature_maps=False):
    total_train_loss = 0
    for MRI_image, PET_image, labels in train_loader:
        MRI_image = MRI_image.to(device)
        PET_image = PET_image.to(device)
        labels = labels.to(device)
        # Clear gradients
        optimiser.zero_grad()
        
        # Forward propagation
        # feature_map indicates whether the feature map prior final layer is also included in the outputs of the model
        if feature_maps:
            outputs = model((MRI_image, PET_image))[1]
        else:
            outputs = model((MRI_image, PET_image))
        
        # Calculate loss
        loss = error(outputs.flatten(), labels)
        total_train_loss += loss.item()
        
        # Calculating gradients
        loss.backward()

        # Update parameters
        optimiser.step()
        scheduler.step()
        
        # Clear GPU Cache
        torch.cuda.empty_cache()
        gc.collect()
        
        # Calculate accuracy
        preds = outputs.flatten().round()
    print("Average Training Loss", total_train_loss/len(train_loader.dataset))

def validate_model(model, val_loader, error, feature_maps=False):
    correct_predictions_val = 0
    total_val_loss = 0
    with torch.no_grad():
        for MRI_image, PET_image, labels in val_loader:
            MRI_image = MRI_image.to(device)
            PET_image = PET_image.to(device)
            labels = labels.to(device)

            # Forward propagation
            if feature_maps:
                pred = model((MRI_image, PET_image))[1]
            else:
                pred = model((MRI_image, PET_image))
            
            # Calculate loss
            loss = error(pred.flatten(), labels)
            total_val_loss += loss.item()

            # Clear GPU Cache
            torch.cuda.empty_cache()
            gc.collect()

            # Calculate accuracy
            preds = pred.flatten().round()
            #print(pred)
            correct_predictions_val += torch.sum(preds == labels).item()
        print("Validation Accuracy:", correct_predictions_val/len(val_loader.dataset))
        print("Average Validation Loss:", total_val_loss/len(val_loader.dataset))
    return total_val_loss/len(val_loader.dataset)
        
def evaluate_model(model, test_loader, error, feature_maps=False):
    correct_predictions_test = 0
    preds = []
    labels = []
    with torch.no_grad():
        for MRI_image, PET_image, label in test_loader:
            MRI_image = MRI_image.to(device)
            PET_image = PET_image.to(device)
            
            labels.append(label.cpu())
            
            # Forward propagation
            if feature_maps:
                pred = model((MRI_image, PET_image))[1]
            else:
                pred = model((MRI_image, PET_image))

            # Clear GPU Cache
            torch.cuda.empty_cache()
            gc.collect()

            # Calculate accuracy
            pred = pred.flatten().round()
            preds.append(pred.cpu())
            correct_predictions_test += torch.sum(pred.cpu() == label.cpu()).item()
    print("Test Accuracy:", correct_predictions_test/len(test_loader.dataset))    
    print("Recall:", recall_score(torch.cat(labels), torch.cat(preds)))
    print("Precision:", precision_score(torch.cat(labels), torch.cat(preds)))
    print(preds)
    print(labels)

## Stage 1: Patch Feature Extraction

### Load MRI patch models for each patch location

In [12]:
# Reload weights for rerunning
MRI_models = []
for i in range(27):
    patch_model = ResNetV2.generate_model(
        model_depth=10,
        n_classes=1,
        n_input_channels=1,
        shortcut_type='B',
        conv1_t_size=7,
        conv1_t_stride=2,
        no_max_pool=False,
        widen_factor=1.0).to(device)
    patch_model.load_state_dict(torch.load(f"./patch_temp/PATCH_TEMP{i}"))
    MRI_models.append(patch_model)
    

### Load PET patch Models for each patch location

In [13]:
# Reload weights for rerunning
PET_models = []
for i in range(27):
    patch_model = ResNetV2.generate_model(
        model_depth=10,
        n_classes=1,
        n_input_channels=1,
        shortcut_type='B',
        conv1_t_size=7,
        conv1_t_stride=2,
        no_max_pool=False,
        widen_factor=1.0).to(device)
    patch_model.load_state_dict(torch.load(f"./patch_temp/PET_PATCH_TEMP{i}"))
    PET_models.append(patch_model)
    

## Stage 2 :Multimodal Attention Models

The models trained below are at the patch level. A model is trained for each patch location. Feature maps from the trained MRI and PET patch models at the same patch location are combined for classification.

In [14]:
# Define multimodal patch model
# This combines the features from the individual MRI and PET patch models at a patch location

class multimodal_patch_CNN(nn.Module):
    def __init__(self, patch_models):
        super().__init__()
        self.patch_models = nn.ModuleList(patch_models)
        self.drop= nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(200, 100) 
        self.rel1 = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm1d(100)
        
        self.fc2 = nn.Linear(100, 50) 
        self.rel2 = nn.ELU()
        self.batch_norm2 = nn.BatchNorm1d(50)
        
        self.fc3 = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        patch_outputs = []
        for i in range(2):
            patch_output = self.patch_models[i](x[i])[0]  
            patch_outputs.append(self.drop(patch_output))
        output = torch.cat(patch_outputs, dim=1)
        output = self.rel1(self.fc1(output))
        features = self.batch_norm1(output)
        output = self.sigmoid(self.fc3(features))
        return features, output

In [15]:
# Define multimodal patch model wtih Attention based mechanism
# This combines the features from the individual MRI and PET patch models at a patch location

class multimodal_patch_CNN_attention(nn.Module):
    def __init__(self, patch_models):
        super().__init__()
        self.patch_models = nn.ModuleList(patch_models)
        self.drop= nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(200, 100) 
        self.rel1 = nn.ReLU()
        self.batch_norm1 = nn.BatchNorm1d(100)
        
        self.fc2 = nn.Linear(100, 50) 
        self.rel2 = nn.ELU()
        self.batch_norm2 = nn.BatchNorm1d(50)
        self.att = nn.MultiheadAttention(embed_dim=100, num_heads=4,dropout=0.4)
        self.fc3 = nn.Linear(200, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        patch_outputs = []
        #attention_patch_outputs = []
        for i in range(2):
            patch_output = self.patch_models[i](x[i])[0]  
            patch_outputs.append(self.drop(patch_output))
    
        x = torch.stack(patch_outputs)
        x, _ = self.att(x,x,x)
        x = x.permute(1,0,2)
        features = x.reshape(x.shape[0],200)
        output = self.sigmoid(self.fc3(features))
        return features, output

### Multimodal with Attention

In [25]:
# Set seeds
torch.manual_seed(101)
torch.cuda.manual_seed(101)
random.seed(101)
np.random.seed(101)


MRI_PET_patch_models = []

# For each patch location, concatenate outputs of trained patch models for each modality and learn local features
for i in range(27):
    MRI_patch_model = MRI_models[i]
    PET_patch_model = PET_models[i]
    MRI_PET_patch_model = multimodal_patch_CNN_attention((MRI_patch_model,PET_patch_model)).to(device)
    
    # Freeze MRI and PET patch model layers (we only want the feature maps)
    for name, param in MRI_patch_model.named_parameters():
            param.requires_grad = False 
            
    for name, param in PET_patch_model.named_parameters():
            param.requires_grad = False 
    
    
    # Instantiate dataloaders
    MRI_PET_train_dataloaders = patch_train_loaders_MRI_PET[i]
    MRI_PET_val_dataloaders = patch_val_loaders_MRI_PET[i]
    MRI_PET_test_dataloaders = patch_test_loaders_MRI_PET[i]
    
    best_val_loss = np.inf
    patience = 15
    no_improvement = 0
    
    # Binary Cross Entropy Loss
    error = nn.BCELoss()

    # SGD Optimizer
    optimiser = SGD(MRI_PET_patch_model.parameters(), lr=0.0001, momentum=0.99)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimiser, start_factor=1.0, end_factor=0.1, total_iters=10)
    
    for epoch in range(500):
        print(f"----------------------------EPOCH {epoch} PATCH {i}-------------------------------")
        MRI_PET_patch_model.train()
        train_model(MRI_PET_patch_model, MRI_PET_train_dataloaders, optimiser, error, scheduler, True)

        MRI_PET_patch_model.eval()
        val_loss = validate_model(MRI_PET_patch_model, MRI_PET_val_dataloaders, error, True)
        
        # Save model weights if improvement seen
        # Otherwise stop model training if there is no improvement in loss after "patience" number of runs
        if val_loss <= best_val_loss:
            best_val_loss = val_loss
            no_improvement = 0
            torch.save(MRI_PET_patch_model.state_dict(), f"./patch_temp/multimodal_PATCH_TEMP_attention{i}")
        else:
            no_improvement += 1
            if no_improvement <= patience:
                continue
            else:
                print(f"BEST VAL ACC: {best_val_loss}")
                break
         
    print("----------------------------TEST RESULTS-------------------------------")
    MRI_PET_patch_model.load_state_dict(torch.load(f"./patch_temp/multimodal_PATCH_TEMP_attention{i}"))
    MRI_PET_patch_model.eval()
    evaluate_model(MRI_PET_patch_model, MRI_PET_test_dataloaders, error, True)
    MRI_PET_patch_models.append(MRI_PET_patch_model)

----------------------------EPOCH 0 PATCH 0-------------------------------
Average Training Loss 0.046272377620954985
Validation Accuracy: 0.4878048780487805
Average Validation Loss: 0.05150613116055
----------------------------EPOCH 1 PATCH 0-------------------------------
Average Training Loss 0.045924445400472545
Validation Accuracy: 0.4878048780487805
Average Validation Loss: 0.05143289013606746
----------------------------EPOCH 2 PATCH 0-------------------------------
Average Training Loss 0.04530033469200134
Validation Accuracy: 0.4878048780487805
Average Validation Loss: 0.05130598893979701
----------------------------EPOCH 3 PATCH 0-------------------------------
Average Training Loss 0.04505757120300512
Validation Accuracy: 0.4878048780487805
Average Validation Loss: 0.051160318095509597
----------------------------EPOCH 4 PATCH 0-------------------------------
Average Training Loss 0.04461041935643212
Validation Accuracy: 0.4878048780487805
Average Validation Loss: 0.05099529

Average Training Loss 0.023580997754804423
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04565386365099651
----------------------------EPOCH 41 PATCH 0-------------------------------
Average Training Loss 0.023899576153422964
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.045269384616758765
----------------------------EPOCH 42 PATCH 0-------------------------------
Average Training Loss 0.0231407105189855
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04524412387754859
----------------------------EPOCH 43 PATCH 0-------------------------------
Average Training Loss 0.022323761257480403
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.045381596902521644
----------------------------EPOCH 44 PATCH 0-------------------------------
Average Training Loss 0.021055278536237655
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04510545875968003
----------------------------EPOCH 45 PATCH 0------------

Average Training Loss 0.042039020994647604
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.05010557174682617
----------------------------EPOCH 13 PATCH 1-------------------------------
Average Training Loss 0.0422254916830141
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.05005763507470852
----------------------------EPOCH 14 PATCH 1-------------------------------
Average Training Loss 0.04200727297145812
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.05001290978454962
----------------------------EPOCH 15 PATCH 1-------------------------------
Average Training Loss 0.041623757755170104
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.04995088897100309
----------------------------EPOCH 16 PATCH 1-------------------------------
Average Training Loss 0.04151722361318401
Validation Accuracy: 0.5121951219512195
Average Validation Loss: 0.049904116770116295
----------------------------EPOCH 17 PATCH 1---------------

Average Training Loss 0.03497495221309974
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04801954874178258
----------------------------EPOCH 54 PATCH 1-------------------------------
Average Training Loss 0.0344663572604539
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04794392498528085
----------------------------EPOCH 55 PATCH 1-------------------------------
Average Training Loss 0.0346752580438481
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04795718338431382
----------------------------EPOCH 56 PATCH 1-------------------------------
Average Training Loss 0.034316205404332424
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.047820220633250914
----------------------------EPOCH 57 PATCH 1-------------------------------
Average Training Loss 0.034031741321086884
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04781971035934076
----------------------------EPOCH 58 PATCH 1----------------

Average Training Loss 0.02666524913711626
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04743323675016078
----------------------------EPOCH 95 PATCH 1-------------------------------
Average Training Loss 0.02692718345855103
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.04730354576576047
----------------------------EPOCH 96 PATCH 1-------------------------------
Average Training Loss 0.026924150644755753
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.047276601558778344
----------------------------EPOCH 97 PATCH 1-------------------------------
Average Training Loss 0.025962535108699173
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04719491266622776
----------------------------EPOCH 98 PATCH 1-------------------------------
Average Training Loss 0.026449195552067678
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04721089252611486
----------------------------EPOCH 99 PATCH 1-------------

...

Average Training Loss 0.03386692116495039
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04557304992908385
----------------------------EPOCH 70 PATCH 25-------------------------------
Average Training Loss 0.03347712694132914
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04554759438444928
----------------------------EPOCH 71 PATCH 25-------------------------------
Average Training Loss 0.03389395284848135
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04542371121848502
----------------------------EPOCH 72 PATCH 25-------------------------------
Average Training Loss 0.03308946462195428
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04543011653714064
----------------------------EPOCH 73 PATCH 25-------------------------------
Average Training Loss 0.03328469694882143
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.045319230091281054
----------------------------EPOCH 74 PATCH 25-----------

Average Training Loss 0.027136946432903163
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.044333979850862085
----------------------------EPOCH 110 PATCH 25-------------------------------
Average Training Loss 0.027267536911808075
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04439855348773119
----------------------------EPOCH 111 PATCH 25-------------------------------
Average Training Loss 0.028364042644617986
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.044372805734960045
----------------------------EPOCH 112 PATCH 25-------------------------------
Average Training Loss 0.027165598129151296
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.044432022222658484
----------------------------EPOCH 113 PATCH 25-------------------------------
Average Training Loss 0.027289091258263978
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04443822255948695
----------------------------EPOCH 114 PATCH 2

Average Training Loss 0.040938089250541126
Validation Accuracy: 0.5365853658536586
Average Validation Loss: 0.04883083919199502
----------------------------EPOCH 31 PATCH 26-------------------------------
Average Training Loss 0.040690730585426584
Validation Accuracy: 0.5365853658536586
Average Validation Loss: 0.04879365897760159
----------------------------EPOCH 32 PATCH 26-------------------------------
Average Training Loss 0.04045896048917145
Validation Accuracy: 0.5365853658536586
Average Validation Loss: 0.04870862931739993
----------------------------EPOCH 33 PATCH 26-------------------------------
Average Training Loss 0.04035874395096888
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04863425580466666
----------------------------EPOCH 34 PATCH 26-------------------------------
Average Training Loss 0.04043470188731053
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.048579800419691135
----------------------------EPOCH 35 PATCH 26---------

Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.045722468597132984
----------------------------EPOCH 71 PATCH 26-------------------------------
Average Training Loss 0.03471743248280932
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.045617045425787206
----------------------------EPOCH 72 PATCH 26-------------------------------
Average Training Loss 0.034409138694649836
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.045614188764153456
----------------------------EPOCH 73 PATCH 26-------------------------------
Average Training Loss 0.03458737038442346
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.045505687957856714
----------------------------EPOCH 74 PATCH 26-------------------------------
Average Training Loss 0.03404462655059627
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04539187361554402
----------------------------EPOCH 75 PATCH 26-------------------------------
Average Training 

Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04348191255476416
----------------------------EPOCH 111 PATCH 26-------------------------------
Average Training Loss 0.028705897267724646
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04344972750035728
----------------------------EPOCH 112 PATCH 26-------------------------------
Average Training Loss 0.027969761461508078
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04349294959045038
----------------------------EPOCH 113 PATCH 26-------------------------------
Average Training Loss 0.028123473962310883
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04350928562443431
----------------------------EPOCH 114 PATCH 26-------------------------------
Average Training Loss 0.028157322072103374
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04346885623001471
----------------------------EPOCH 115 PATCH 26-------------------------------
Average Train

### Mutlimodal without attention

In [16]:
# Set Seeds
torch.manual_seed(101)
torch.cuda.manual_seed(101)
random.seed(101)
np.random.seed(101)


MRI_PET_patch_models = []

# For each patch location, concatenate outputs of trained patch models for each modality and learn local features
for i in range(27):
    MRI_patch_model = MRI_models[i]
    PET_patch_model = PET_models[i]
    MRI_PET_patch_model = multimodal_patch_CNN((MRI_patch_model,PET_patch_model)).to(device)
    
    # Freeze MRI and PET patch model layers (we only want the feature maps)
    for name, param in MRI_patch_model.named_parameters():
            param.requires_grad = False 
            
    for name, param in PET_patch_model.named_parameters():
            param.requires_grad = False 
    
    # Instantiate dataloaders
    MRI_PET_train_dataloaders = patch_train_loaders_MRI_PET[i]
    MRI_PET_val_dataloaders = patch_val_loaders_MRI_PET[i]
    MRI_PET_test_dataloaders = patch_test_loaders_MRI_PET[i]
    
    best_val_loss = np.inf
    patience = 15
    no_improvement = 0
    
    # Binary Cross Entropy Loss
    error = nn.BCELoss()

    # SGD Optimizer
    optimiser = SGD(MRI_PET_patch_model.parameters(), lr=0.0001, momentum=0.99)
    scheduler = torch.optim.lr_scheduler.LinearLR(optimiser, start_factor=1.0, end_factor=0.1, total_iters=10)
    
    for epoch in range(500):
        print(f"----------------------------EPOCH {epoch} PATCH {i}-------------------------------")
        MRI_PET_patch_model.train()
        train_model(MRI_PET_patch_model, MRI_PET_train_dataloaders, optimiser, error, scheduler, True)

        MRI_PET_patch_model.eval()
        val_loss = validate_model(MRI_PET_patch_model, MRI_PET_val_dataloaders, error, True)
        
        # Save model weights if improvement seen
        # Otherwise stop model training if there is no improvement in loss after "patience" number of runs
        if val_loss <= best_val_loss:
            best_val_loss = val_loss
            no_improvement = 0
            torch.save(MRI_PET_patch_model.state_dict(), f"./patch_temp/multimodal_PATCH_TEMP{i}")
        else:
            no_improvement += 1
            if no_improvement <= patience:
                continue
            else:
                print(f"BEST VAL ACC: {best_val_loss}")
                break
         
    print("----------------------------TEST RESULTS-------------------------------")
    MRI_PET_patch_model.load_state_dict(torch.load(f"./patch_temp/multimodal_PATCH_TEMP{i}"))
    MRI_PET_patch_model.eval()
    evaluate_model(MRI_PET_patch_model, MRI_PET_test_dataloaders, error, True)
    MRI_PET_patch_models.append(MRI_PET_patch_model)

----------------------------EPOCH 0 PATCH 0-------------------------------
Average Training Loss 0.046653531369615774
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.049145051618901696
----------------------------EPOCH 1 PATCH 0-------------------------------
Average Training Loss 0.04149744390952782
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04714423708799409
----------------------------EPOCH 2 PATCH 0-------------------------------
Average Training Loss 0.03467043330434893
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04541407561883694
----------------------------EPOCH 3 PATCH 0-------------------------------
Average Training Loss 0.026735178332348338
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.044429315299522584
----------------------------EPOCH 4 PATCH 0-------------------------------
Average Training Loss 0.021483917491602115
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.044

Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.05203052046822339
----------------------------EPOCH 17 PATCH 1-------------------------------
Average Training Loss 0.019477558826081088
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.05257413300072274
----------------------------EPOCH 18 PATCH 1-------------------------------
Average Training Loss 0.02123911318476083
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.052279433099234975
----------------------------EPOCH 19 PATCH 1-------------------------------
Average Training Loss 0.018683635248023956
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.054183374090892515
----------------------------EPOCH 20 PATCH 1-------------------------------
Average Training Loss 0.020253878850184502
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.052856056428537135
----------------------------EPOCH 21 PATCH 1-------------------------------
Average Training Loss

Average Training Loss 0.02345179293121471
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04718594507473271
----------------------------EPOCH 7 PATCH 3-------------------------------
Average Training Loss 0.020745083506478638
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.04767885949553513
----------------------------EPOCH 8 PATCH 3-------------------------------
Average Training Loss 0.019042784042778562
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.04821302469183759
----------------------------EPOCH 9 PATCH 3-------------------------------
Average Training Loss 0.017599522731587536
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04861129202493807
----------------------------EPOCH 10 PATCH 3-------------------------------
Average Training Loss 0.01570639551663008
Validation Accuracy: 0.5853658536585366
Average Validation Loss: 0.0499592001845197
----------------------------EPOCH 11 PATCH 3------------------

Average Training Loss 0.0238653371935008
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.05352873918486804
BEST VAL ACC: 0.048598687823225815
----------------------------TEST RESULTS-------------------------------
Test Accuracy: 0.8048780487804879
Recall: 0.6666666666666666
Precision: 0.9333333333333333
[tensor([1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]), tensor([0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]), tensor([0., 0., 0., 0., 1., 1., 1., 1., 0.])]
[tensor([1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.]), tensor([0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0.]), tensor([0., 0., 0., 1., 1., 1., 1., 1., 0.])]
----------------------------EPOCH 0 PATCH 5-------------------------------
Average Training Loss 0.04137889821021283
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.05049032409016679
----------------------------EPOCH 1 PATCH 5-------------------------------
Average T

Average Training Loss 0.01345612936210437
Validation Accuracy: 0.7317073170731707
Average Validation Loss: 0.04543773575526912
----------------------------EPOCH 13 PATCH 6-------------------------------
Average Training Loss 0.013246084151209378
Validation Accuracy: 0.7317073170731707
Average Validation Loss: 0.04532000640543496
----------------------------EPOCH 14 PATCH 6-------------------------------
Average Training Loss 0.01124993026195491
Validation Accuracy: 0.7317073170731707
Average Validation Loss: 0.04572672378726122
----------------------------EPOCH 15 PATCH 6-------------------------------
Average Training Loss 0.011107133273951342
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04603245200180426
----------------------------EPOCH 16 PATCH 6-------------------------------
Average Training Loss 0.011385197735956459
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.0462440121464613
----------------------------EPOCH 17 PATCH 6---------------

...

Average Training Loss 0.05150498669655597
Validation Accuracy: 0.43902439024390244
Average Validation Loss: 0.05149905419931179
----------------------------EPOCH 1 PATCH 19-------------------------------
Average Training Loss 0.04797954368786734
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04956214747777799
----------------------------EPOCH 2 PATCH 19-------------------------------
Average Training Loss 0.04151868673621631
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04699830165723475
----------------------------EPOCH 3 PATCH 19-------------------------------
Average Training Loss 0.03524597373897912
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.04550470375433201
----------------------------EPOCH 4 PATCH 19-------------------------------
Average Training Loss 0.030769491354461578
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04468550187785451
----------------------------EPOCH 5 PATCH 19---------------

Average Training Loss 0.030677397231586644
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.05106640588946459
----------------------------EPOCH 17 PATCH 20-------------------------------
Average Training Loss 0.028573045049045908
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.05165505554617905
----------------------------EPOCH 18 PATCH 20-------------------------------
Average Training Loss 0.02803543573398082
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.052220312560476906
----------------------------EPOCH 19 PATCH 20-------------------------------
Average Training Loss 0.026312649188960185
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.05175894789579438
----------------------------EPOCH 20 PATCH 20-------------------------------
Average Training Loss 0.027612599070932043
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.05137979984283447
----------------------------EPOCH 21 PATCH 20-------

Average Training Loss 0.02857055337946923
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.048156146596117715
----------------------------EPOCH 8 PATCH 22-------------------------------
Average Training Loss 0.027810715810685862
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.04909088844206275
----------------------------EPOCH 9 PATCH 22-------------------------------
Average Training Loss 0.027687622875463766
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.049238710868649366
----------------------------EPOCH 10 PATCH 22-------------------------------
Average Training Loss 0.026781153391863478
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.05048201723796565
----------------------------EPOCH 11 PATCH 22-------------------------------
Average Training Loss 0.0243155053038089
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.050583999331404526
----------------------------EPOCH 12 PATCH 22---------

Average Training Loss 0.03140748597559382
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.048330071495800486
BEST VAL ACC: 0.04649055294874238
----------------------------TEST RESULTS-------------------------------
Test Accuracy: 0.6829268292682927
Recall: 0.5238095238095238
Precision: 0.7857142857142857
[tensor([1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.]), tensor([0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0.]), tensor([0., 0., 0., 1., 1., 0., 1., 1., 1.])]
[tensor([1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0.]), tensor([0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0.]), tensor([0., 0., 0., 1., 1., 1., 1., 1., 0.])]
----------------------------EPOCH 0 PATCH 24-------------------------------
Average Training Loss 0.051422397377061065
Validation Accuracy: 0.34146341463414637
Average Validation Loss: 0.051974844641801785
----------------------------EPOCH 1 PATCH 24-------------------------------
Ave

Average Training Loss 0.023993303541277277
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.045705440567760935
----------------------------EPOCH 12 PATCH 25-------------------------------
Average Training Loss 0.024800153571318408
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.046370126852175085
----------------------------EPOCH 13 PATCH 25-------------------------------
Average Training Loss 0.022762024530866107
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.047053343639141175
----------------------------EPOCH 14 PATCH 25-------------------------------
Average Training Loss 0.02378240032274215
Validation Accuracy: 0.7073170731707317
Average Validation Loss: 0.04712112356976765
----------------------------EPOCH 15 PATCH 25-------------------------------
Average Training Loss 0.024421406787682752
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.04762030083958695
----------------------------EPOCH 16 PATCH 25-----

# Stage 3. Patch Fusion

In [17]:
# Stacking all 27 patch based datasets to create a single subject level dataset
# We will now have a subject level dataset where each row is a subject and their 27 patch images
# e.g row 1 would be: [patch_1,....,patch_27] for subject 1

all_patch_train_MRI = torch.stack([dataset.tensors[0] for dataset in patch_train_datasets_MRI_PET], dim=1)
all_patch_val_MRI = torch.stack([dataset.tensors[0] for dataset in patch_val_datasets_MRI_PET], dim=1)
all_patch_test_MRI = torch.stack([dataset.tensors[0] for dataset in patch_test_datasets_MRI_PET], dim=1)

all_patch_train_PET = torch.stack([dataset.tensors[1] for dataset in patch_train_datasets_MRI_PET], dim=1)
all_patch_val_PET = torch.stack([dataset.tensors[1] for dataset in patch_val_datasets_MRI_PET], dim=1)
all_patch_test_PET = torch.stack([dataset.tensors[1] for dataset in patch_test_datasets_MRI_PET], dim=1)

all_patch_train_MRI_PET_dataset  = torch.utils.data.TensorDataset(all_patch_train_MRI, all_patch_train_PET, train_y)
all_patch_val_MRI_PET_dataset = torch.utils.data.TensorDataset(all_patch_val_MRI, all_patch_val_PET, val_y)
all_patch_test_MRI_PET_dataset = torch.utils.data.TensorDataset(all_patch_test_MRI, all_patch_test_PET, test_y)

In [18]:
batch_size = 10
all_patch_train_dataloader_MRI_PET = torch.utils.data.DataLoader(all_patch_train_MRI_PET_dataset, 
                                                                batch_size = batch_size, shuffle = True)
all_patch_val_dataloader_MRI_PET = torch.utils.data.DataLoader(all_patch_val_MRI_PET_dataset, 
                                                                batch_size = batch_size, shuffle = True)
all_patch_test_dataloader_MRI_PET = torch.utils.data.DataLoader(all_patch_test_MRI_PET_dataset, 
                                                                batch_size = batch_size, shuffle = False)



In [43]:
# Define our final multimodal attention-based patch network
class multimodal_CNN(nn.Module):
    def __init__(self, MRI_PET_patch_models):
        super().__init__()
        self.MRI_PET_patch_models = nn.ModuleList(MRI_PET_patch_models)
        self.drop= nn.Dropout(p=0.4)
        self.fc3 = nn.Linear(5400, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        patch_outputs = []
        for i in range(27):
            mri_image = x[0][:, i]
            pet_image = x[1][:, i]
            patch_output, _ = self.MRI_PET_patch_models[i]((mri_image, pet_image))
            patch_outputs.append(self.drop(patch_output))
        x = torch.cat(patch_outputs, dim=1)
        x = self.sigmoid(self.fc3(x))
        return x

### Final Multimodal Patch Model with attention

In [36]:
# Reload attention weights for rerunning
MRI_PET_patch_models = []
for i in range(27):
    MRI_PET_patch_model = multimodal_patch_CNN_attention((MRI_models[i],PET_models[i])).to(device)
    MRI_PET_patch_model.load_state_dict(torch.load(f"./patch_temp/multimodal_PATCH_TEMP_attention{i}"))                                           
    MRI_PET_patch_models.append(MRI_PET_patch_model)
    
MRI_patch_model = multimodal_CNN(MRI_PET_patch_models).to(device)

# Freeze layers of the patch models (we only want the feature maps)
for patch_model in MRI_patch_model.MRI_PET_patch_models:
    for name, param in patch_model.named_parameters():
        param.requires_grad = False 

# Set seeds
torch.manual_seed(101)
torch.cuda.manual_seed(101)
random.seed(101)
np.random.seed(101)

# Binary Cross Entropy Loss
error = nn.BCELoss()

# SGD Optimizer
optimiser = SGD(MRI_patch_model.parameters(), lr=0.0001, momentum=0.9)
scheduler = lr_scheduler.LinearLR(optimiser, start_factor=1.0, end_factor=0.1, total_iters=10)


best_val_loss = np.inf
patience = 20
no_improvement = 0

for epoch in range(1000):
    print(f"----------------------------EPOCH {epoch}-------------------------------")
    MRI_patch_model.train()
    train_model(MRI_patch_model, all_patch_train_dataloader_MRI_PET, optimiser, error, scheduler)
    
    MRI_patch_model.eval()
    avg_val_loss = validate_model(MRI_patch_model, all_patch_val_dataloader_MRI_PET, error)
    
    # Save model weights if improvement seen
    # Otherwise stop model training if there is no improvement in loss after "patience" number of runs
    if avg_val_loss <= best_val_loss:
        best_val_loss = avg_val_loss
        no_improvement = 0
        torch.save(MRI_patch_model.state_dict(), "./trained_models_2/MULTIMODAL_MODEL_MODAL_ATTENTION")
    else:
        no_improvement += 1
        if no_improvement <= patience:
            continue
        else:
            print(f"BEST VAL LOSS: {best_val_loss}")
            break
            
print("----------------------------TEST RESULTS-------------------------------")
MRI_patch_model.load_state_dict(torch.load("./trained_models_2/MULTIMODAL_MODEL_MODAL_ATTENTION"))
MRI_patch_model.eval()
evaluate_model(MRI_patch_model, all_patch_test_dataloader_MRI_PET, error)

----------------------------EPOCH 0-------------------------------
Average Training Loss 0.06957613017226828
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.08323001280063536
----------------------------EPOCH 1-------------------------------
Average Training Loss 0.06542665120519575
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.08100275295536692
----------------------------EPOCH 2-------------------------------
Average Training Loss 0.06215867620022571
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.07506222259707568
----------------------------EPOCH 3-------------------------------
Average Training Loss 0.0587429804880111
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.07791784914528452
----------------------------EPOCH 4-------------------------------
Average Training Loss 0.05577169248803717
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.08086491648743792
----------------------------EPO

Average Training Loss 0.02452806300926404
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.06684592438907158
----------------------------EPOCH 43-------------------------------
Average Training Loss 0.02915794249685084
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.0702993557220552
----------------------------EPOCH 44-------------------------------
Average Training Loss 0.026022478938102722
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.09396970199375618
----------------------------EPOCH 45-------------------------------
Average Training Loss 0.02617792951584351
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.06987445165471333
----------------------------EPOCH 46-------------------------------
Average Training Loss 0.024015497370455108
Validation Accuracy: 0.6585365853658537
Average Validation Loss: 0.069707073816439
----------------------------EPOCH 47-------------------------------
Average Training Loss 0.02

### Final Multimodal Patch Model with no attention

In [20]:
# Reload no attention weights for rerunning
MRI_PET_patch_models = []
for i in range(27):
    MRI_PET_patch_model = multimodal_patch_CNN((MRI_models[i],PET_models[i])).to(device)
    MRI_PET_patch_model.load_state_dict(torch.load(f"./patch_temp/multimodal_PATCH_TEMP{i}"))                                           
    MRI_PET_patch_models.append(MRI_PET_patch_model)
    
MRI_patch_model = multimodal_CNN(MRI_PET_patch_models).to(device)

# Freeze layers of the patch models (we only want the feature maps)
for patch_model in MRI_patch_model.MRI_PET_patch_models:
    for name, param in patch_model.named_parameters():
        param.requires_grad = False 

# Set seeds
torch.manual_seed(101)
torch.cuda.manual_seed(101)
random.seed(101)
np.random.seed(101)

# Binary Cross Entropy Loss
error = nn.BCELoss()

# SGD Optimizer
optimiser = SGD(MRI_patch_model.parameters(), lr=0.0001, momentum=0.9)
scheduler = lr_scheduler.LinearLR(optimiser, start_factor=1.0, end_factor=0.1, total_iters=10)


best_val_loss = np.inf
patience = 20
no_improvement = 0

for epoch in range(1000):
    print(f"----------------------------EPOCH {epoch}-------------------------------")
    MRI_patch_model.train()
    train_model(MRI_patch_model, all_patch_train_dataloader_MRI_PET, optimiser, error, scheduler)
    
    MRI_patch_model.eval()
    avg_val_loss = validate_model(MRI_patch_model, all_patch_val_dataloader_MRI_PET, error)
    
    # Save model weights if improvement seen
    # Otherwise stop model training if there is no improvement in loss after "patience" number of runs
    if avg_val_loss <= best_val_loss:
        best_val_loss = avg_val_loss
        no_improvement = 0
        torch.save(MRI_patch_model.state_dict(), "./trained_models_2/MULTIMODAL_MODEL_4")
    else:
        no_improvement += 1
        if no_improvement <= patience:
            continue
        else:
            print(f"BEST VAL LOSS: {best_val_loss}")
            break
            
print("----------------------------TEST RESULTS-------------------------------")
MRI_patch_model.load_state_dict(torch.load("./trained_models_2/MULTIMODAL_MODEL_4"))
MRI_patch_model.eval()
evaluate_model(MRI_patch_model, all_patch_test_dataloader_MRI_PET, error)

----------------------------EPOCH 0-------------------------------
Average Training Loss 0.06529779805511725
Validation Accuracy: 0.6341463414634146
Average Validation Loss: 0.0864112319015875
----------------------------EPOCH 1-------------------------------
Average Training Loss 0.055793249521587714
Validation Accuracy: 0.6097560975609756
Average Validation Loss: 0.07096719014935376
----------------------------EPOCH 2-------------------------------
Average Training Loss 0.045468686301200115
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.06271800834958147
----------------------------EPOCH 3-------------------------------
Average Training Loss 0.04139709152037003
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.06574072198169988
----------------------------EPOCH 4-------------------------------
Average Training Loss 0.034925995852615015
Validation Accuracy: 0.6829268292682927
Average Validation Loss: 0.08918767585986997
----------------------------